In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import os 
os.environ['OMP_NUM_THREADS'] = '4'
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split
TRAIN_SAMPLE_SIZE = 800000
import pprint

C:\Users\hnpathak\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# data is divided into positive negative samples and 7 gb file of negatives is further divided into 250 splits. 
# $ split -n l/250 negatives.csv
print('Negative samples')
data_neg = pd.read_csv('data/negatives/xaa', header=None)
print(data_neg.head())
print(data_neg.shape)
print('\n Positive samples')
data_pos = pd.read_csv('data/positives/positives.csv',header=None)
print(data_pos.head())
print(data_pos.shape)

Negative samples
        0   1  2   3    4                    5   6  7
0   83230   3  1  13  379  2017-11-06 14:32:21 NaN  0
1   17357   3  1  19  379  2017-11-06 14:33:34 NaN  0
2   35810   3  1  13  379  2017-11-06 14:34:12 NaN  0
3   45745  14  1  13  478  2017-11-06 14:34:52 NaN  0
4  161007   3  1  13  379  2017-11-06 14:35:08 NaN  0
(737196, 8)

 Positive samples
        0   1  2   3    4                    5                    6  7
0  204158  35  1  13   21  2017-11-06 15:41:07  2017-11-07 08:17:19  1
1   29692   9  1  22  215  2017-11-06 16:00:02  2017-11-07 10:05:22  1
2   64516  35  1  13   21  2017-11-06 16:00:02  2017-11-06 23:40:50  1
3  172429  35  1  46  274  2017-11-06 16:00:03  2017-11-07 00:55:29  1
4  199085  35  1  13  274  2017-11-06 16:00:04  2017-11-06 23:04:54  1
(456846, 8)


In [3]:
# Build x_train, y_tarin, x_test, y_test
data_combined =  pd.concat([data_pos, data_neg])
data_combined = data_combined.sample(frac=1)
print('data_combined: ', data_combined.shape)


data_combined:  (1194042, 8)


In [17]:
data_f = data_combined.iloc[:,0:5]
data_f.astype(float)
labels = data_combined.iloc[:,7:8]
labels.astype(float)
type(labels)
#n_values = np.max(labels) 
labels_ = np.eye(2)[(labels.values).reshape(-1)]
labels_.shape
# labels = labels.values
# enc = OneHotEncoder()
# enc.fit(labels)
# labels= enc.transform(labels)
# labels.shape
#del data_combined, data_neg, data_pos
# y = np.zeros((data_combined.shape[0], 2))
# y[np.arange(data_combined.shape[0]), y] = 1

(1194042, 2)

In [18]:
x_train, x_test, y_train, y_test = train_test_split(data_f,labels_,test_size = 0.1,random_state = 100)
x_train.astype(float)
x_test.astype(float)
y_train.astype(float)
y_test.astype(float)

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [6]:
Y_LABEL = 'y'                                   			        # Name of the variable to be predicted
#KEYS = [i for i in raw_data.keys().tolist() if i != Y_LABEL]	# Name of predictors
N_INSTANCES = data_f.shape[0]                     			    # Number of instances
N_INPUT = x_train.shape[1]                      			    # Input size
N_CLASSES = 2                                    			    # Number of classes (output size)
TEST_SIZE = 0.1                                    			      # Test set size (% of dataset)
TRAIN_SIZE = int(N_INSTANCES * (1 - TEST_SIZE))     			    # Train size
LEARNING_RATE = 0.001                               			    # Learning rate
TRAINING_EPOCHS = 40                               			    # Number of epochs
BATCH_SIZE = 100                                    			    # Batch size
DISPLAY_STEP = 5                                    			    # Display progress each x epochs
HIDDEN_SIZE = 10          	                                   		      # Number of hidden neurons 256
ACTIVATION_FUNCTION_OUT = tf.nn.sigmoid                          # Last layer act fct
STDDEV = 0.1                                        			    # Standard deviation (for weights random init)
RANDOM_STATE = 100   		

In [7]:
n_input = N_INPUT                   # input n labels
n_hidden_1 = HIDDEN_SIZE            # 1st layer
n_hidden_2 = HIDDEN_SIZE            # 2nd layer
n_hidden_3 = HIDDEN_SIZE            # 3rd layer
n_hidden_4 = HIDDEN_SIZE            # 4th layer
n_classes = N_CLASSES               # output m classes

In [8]:

X = tf.placeholder(tf.float32, [None, n_input],name='X')
y = tf.placeholder(tf.float32, [None, n_classes],name='y')
dropout_keep_prob = tf.placeholder(tf.float32)


weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=STDDEV)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=STDDEV)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3],stddev=STDDEV)),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4],stddev=STDDEV)),
    'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes],stddev=STDDEV)),                                   
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}    
   
def mlp(_X, _weights, _biases, dropout_keep_prob):
    layer1 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])), dropout_keep_prob)
    layer2 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2'])), dropout_keep_prob)
    layer3 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer2, _weights['h3']), _biases['b3'])), dropout_keep_prob)
    layer4 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer3, _weights['h4']), _biases['b4'])), dropout_keep_prob)
    out = ACTIVATION_FUNCTION_OUT(tf.add(tf.matmul(layer4, _weights['out']), _biases['out']))
    return out



In [9]:
pred = mlp(X, weights, biases, dropout_keep_prob)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) # softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE).minimize(cost)

# Accuracy
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for epoch in range(TRAINING_EPOCHS):
        avg_cost = 0.
        total_batch = int(x_train.shape[0] / BATCH_SIZE)
        X_batches = np.array_split(x_train, total_batch)
        Y_batches = np.array_split(y_train, total_batch)
        # Loop over all batches
        for i in range(total_batch):
#             randidx = np.random.randint(int(TRAIN_SIZE), size = BATCH_SIZE)
#             batch_xs = x_tr[randidx, :]
#             batch_ys = labels[randidx, :]
            batch_xs, batch_ys =  X_batches[i], Y_batches[i]
            # Fit using batched data
            sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob: 0.9})
            # Calculate average cost
            avg_cost += sess.run(cost, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})/total_batch
        # Display progress
        if epoch % DISPLAY_STEP == 0:
            print("Epoch: %03d/%03d cost: %.9f" % (epoch, TRAINING_EPOCHS, avg_cost))
            train_acc = sess.run(accuracy, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})
            print("Training accuracy: %.3f" % (train_acc))
    test_acc = sess.run(accuracy, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
    print ("Test accuracy: %.3f" % (test_acc))


Epoch: 000/040 cost: 0.574416064
Training accuracy: 0.720
Epoch: 005/040 cost: 0.544168553
Training accuracy: 0.730
Epoch: 010/040 cost: 0.534956565
Training accuracy: 0.730
Epoch: 015/040 cost: 0.532019346
Training accuracy: 0.710
Epoch: 020/040 cost: 0.529941426
Training accuracy: 0.780
Epoch: 025/040 cost: 0.521214060
Training accuracy: 0.770
Epoch: 030/040 cost: 0.522534023
Training accuracy: 0.800
Epoch: 035/040 cost: 0.522402465
Training accuracy: 0.780
Test accuracy: 0.766
